**Общая информация**

**Срок сдачи:** 13 марта 2017, 06:00 <br\>
**Штраф за опоздание:** -2 балла после 06:00 13 марта, -4 балла после 06:00 20 марта, -6 баллов после 06:00 27 марта

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush

Необходимо в slack создать таск в приватный чат:
/todo Фамилия Имя ссылка на гитхаб @alkhamush
Пример:
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе.

###### Задание 2 (2 балла)
Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn 4 балла для iris и mnist
Для этого используем numpy

###### Задание 3 (2 балла)
Для iris найдите такой параметр n_neighbors, при котором выдаётся наилучший score. Нарисуйте график зависимости score от n_neighbors

###### Задание 3 (2 балла)
Выполнить требования pep8

###### Задание 5 (2 балла)
Описать для чего нужны следующие библиотеки/классы/функции (список будет ниже)


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
from scipy.spatial import distance
%load_ext pycodestyle_magic

In [13]:
%%pycodestyle

class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin, ClassifierMixin):
    
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors
    
    def fit(self, X, y):
        self.X_tr = X.copy()
        self.y_tr = y.copy()
    
    def predict(self, X):
        predictions = []
        for i in X:
            best_indices = []
            for n in range(self.n_neighbors):
                best_distance = distance.euclidean(i, self.X_tr[0])
                best_current_index = 0
                for j in range(1, len(self.X_tr)):
                    dist = distance.euclidean(i, self.X_tr[j])
                    if dist < best_distance and j not in best_indices:
                        best_distance = dist
                        best_current_index = j
                best_indices.append(best_current_index)
            avg = []
            for x in best_indices:
                  avg.append(self.y_tr[x])
            predictions.append(Counter(avg).most_common(1)[0][0])
            
        return predictions
    
    def predictrr(self, X):
        predictions = []
        cathegories = self.y_tr.max()+1
        for i, x in enumerate(X):
            D = self.X_tr - x
            dists = (D**2).sum(axis=1)
            best_indices = np.argsort(dists)[0:self.n_neighbors:]
            
            avg = self.y_tr[best_indices]
            freq = np.zeros(cathegories)
            for k in avg:
                freq[k]+=1
            predictions.append(np.argmax(freq))
            
        return predictions
    
    def predict_proba(self, X):
        cathegories = self.y_tr.max()+1
        predictions = np.zeros((X.shape[0], cathegories))
        for i, x in enumerate(X):
            D = self.X_tr - x
            dists = (D**2).sum(axis=1)
            best_indices = np.argsort(dists)[0:self.n_neighbors:]
            avg = self.y_tr[best_indices]
            freq = np.zeros(cathegories)
            for k in avg:
                freq[k]+=1
            #normalisation
            freq = freq/freq.sum()
            predictions[i] = freq
            
        return predictions
    
    def score(self, X, y):
        answers = self.predict(X)
        correct = 0
        for i in range(X.shape[0]):
            if answers[i] == y[i]:
                correct +=1
        return correct/float(X.shape[0])

**IRIS**

In [14]:
iris = datasets.load_iris()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [16]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [17]:
%time clf.fit(X_train, y_train)

CPU times: user 1.51 ms, sys: 97 µs, total: 1.61 ms
Wall time: 1.36 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [18]:
%time my_clf.fit(X_train, y_train)

CPU times: user 108 µs, sys: 0 ns, total: 108 µs
Wall time: 155 µs


In [19]:
%time clf.predict(X_test)

CPU times: user 2.94 ms, sys: 0 ns, total: 2.94 ms
Wall time: 2.44 ms


array([2, 0, 1, 0, 1, 2, 2, 0, 2, 1, 0, 1, 2, 2, 0])

In [20]:
%time my_clf.predictrr(X_test)

CPU times: user 3.01 ms, sys: 193 µs, total: 3.21 ms
Wall time: 2.49 ms


[2, 0, 1, 0, 1, 2, 2, 0, 2, 1, 0, 1, 2, 2, 0]

In [21]:
%time clf.predict_proba(X_test)

CPU times: user 1.72 ms, sys: 112 µs, total: 1.83 ms
Wall time: 1.53 ms


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [22]:
%time my_clf.predict_proba(X_test)

CPU times: user 3.57 ms, sys: 81 µs, total: 3.65 ms
Wall time: 2.67 ms


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [23]:
clf.score(X_test, y_test)

0.9333333333333333

In [24]:
my_clf.score(X_test, y_test)

0.9333333333333333

In [ ]:
# Задание 3

**MNIST**

In [25]:
mnist = fetch_mldata('MNIST original')

OSError: could not read bytes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.1, stratify=mnist.target)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

In [ ]:
%time clf.predict(X_test)

In [ ]:
%time my_clf.predict(X_test)

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

Задание 5

In [ ]:
# seaborn
# matplotlib
# train_test_split
# Pipelin%load_ext e (from sklearn.pipeline import Pipeline)
# StandardScaler (from sklearn.preprocessing import StandardScaler)
# ClassifierMixin
# NeighborsBase
# KNeighborsMixin
# SupervisedIntegerMixin